In [1]:
from __future__ import print_function
from functools import reduce
import tensorflow as tf
import numpy as np

In [2]:
try:
    import cPickle as pickle
except ImportError:
    import pickle

In [3]:
notMINST_PATH = "./HW/HW1/notMNIST.pickle"

In [4]:
with open(notMINST_PATH) as rf:
    data = pickle.load(rf)
    train_labels = data["train_labels"]
    valid_labels = data["valid_labels"]
    test_labels = data["test_labels"]
    train_dataset = data["train_dataset"]
    valid_dataset = data["valid_dataset"]
    test_dataset = data['test_dataset']
    for k, v in data.items():
        print("{}: {}".format(k, v.shape))
    del data

train_labels: (200000,)
valid_labels: (10000,)
test_labels: (10000,)
test_dataset: (10000, 28, 28)
train_dataset: (200000, 28, 28)
valid_dataset: (10000, 28, 28)


In [5]:
graph_notMINST = tf.Graph()

In [6]:
def one_hot(arr, num_class=10, dtype=np.float32):
    return (np.arange(num_class) == arr[:,None]).astype(dtype)

In [7]:
num_input_pixels = reduce(lambda acc, x: acc*x, train_dataset.shape[1:])

In [8]:
train_dataset = train_dataset.reshape((-1, num_input_pixels))
test_dataset = test_dataset.reshape((-1, num_input_pixels))
valid_dataset = valid_dataset.reshape((-1, num_input_pixels))
train_labels = one_hot(train_labels)
test_labels = one_hot(test_labels)
valid_labels = one_hot(valid_labels)

In [44]:
mini_batch_size = 3000
num_class = 10
with graph_notMINST.as_default():
    
    tf_input_layer = tf.placeholder(dtype=tf.float32, shape=(mini_batch_size, num_input_pixels))
    
    tf_weight_hidden = tf.Variable(tf.truncated_normal(shape=(num_input_pixels, 30)))
    tf_bias_hidden = tf.Variable(tf.truncated_normal(shape=(30,)))
    tf_hidden_layer = tf.matmul(tf_input_layer, tf_weight_hidden) + tf_bias_hidden
    tf_hidden_act = tf.sigmoid(tf_hidden_layer)
    
    tf_weight_output = tf.Variable(tf.truncated_normal(shape=(30, 10)))
    tf_bias_output = tf.Variable(tf.truncated_normal(shape=(10,)))
    tf_logits = tf.matmul(tf_hidden_act, tf_weight_output)
    
    tf_output_label = tf.placeholder(dtype=tf.float32, shape=(mini_batch_size, num_class))
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(tf_logits, tf_output_label))
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_hidden_layer = tf.matmul(tf_test_dataset, tf_weight_hidden) + tf_bias_hidden
    tf_test_hidden_act = tf.sigmoid(tf_test_hidden_layer)
    tf_test_logits = tf.matmul(tf_test_hidden_act, tf_weight_output) + tf_bias_output
    tf_test_output = tf.sigmoid(tf_test_logits)
    
    optimizer = tf.train.GradientDescentOptimizer(2.0).minimize(loss)

In [45]:
def accuracy(predict, target):
    N = predict.shape[0]
    correct_count = np.sum(np.argmax(predict, 1) == np.argmax(target, 1))
    return 100.0 * correct_count / N

In [46]:
num_iteration = 4001
N_train = train_dataset.shape[0]
with tf.Session(graph=graph_notMINST) as session:
    tf.global_variables_initializer().run()
    print("all variables initialized")
    for i in range(num_iteration):
        random_indices = np.random.choice(N_train, mini_batch_size)
        train_batch_data = train_dataset[random_indices]
        train_batch_label = train_labels[random_indices]
        feed_dict = {tf_input_layer:train_batch_data,
                     tf_output_label:train_batch_label}
        
        if (i+1) % 100 == 0:
            _, l, o, predict_test = session.run([optimizer, loss, tf_output_label, tf_test_output], feed_dict=feed_dict)
            print("accuracy (train, batch): {}%".format(accuracy(o, train_batch_label)))
            print("accuracy (test): {}%".format(accuracy(predict_test, test_labels)))
            print("Iteraion {}, loss {}".format(i+1, l))
        else:
            _, l, o = session.run([optimizer, loss, tf_output_label], feed_dict=feed_dict)

all variables initialized
accuracy (train, batch): 100.0%
accuracy (test): 40.08%
Iteraion 100, loss 0.27322024107
accuracy (train, batch): 100.0%
accuracy (test): 55.88%
Iteraion 200, loss 0.222522750497
accuracy (train, batch): 100.0%
accuracy (test): 63.82%
Iteraion 300, loss 0.200016960502
accuracy (train, batch): 100.0%
accuracy (test): 67.85%
Iteraion 400, loss 0.186012506485
accuracy (train, batch): 100.0%
accuracy (test): 71.13%
Iteraion 500, loss 0.179791390896
accuracy (train, batch): 100.0%
accuracy (test): 73.31%
Iteraion 600, loss 0.168522119522
accuracy (train, batch): 100.0%
accuracy (test): 74.99%
Iteraion 700, loss 0.159553557634
accuracy (train, batch): 100.0%
accuracy (test): 76.34%
Iteraion 800, loss 0.152389198542
accuracy (train, batch): 100.0%
accuracy (test): 77.37%
Iteraion 900, loss 0.154616519809
accuracy (train, batch): 100.0%
accuracy (test): 78.04%
Iteraion 1000, loss 0.154568463564
accuracy (train, batch): 100.0%
accuracy (test): 78.64%
Iteraion 1100, los